In [1]:
import os, json, datetime, sys
from operator import attrgetter
from azureml.core import Workspace
from azureml.core import Environment
from azureml.core.authentication import AzureCliAuthentication

from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice, Webservice

In [3]:
with open("./configuration/config.json") as f:
    config = json.load(f)
    
workspace_name = config["workspace_name"]
resource_group = config["resource_group"]
subscription_id = config["subscription_id"]
location = config["location"]


# Get workspace
#ws = Workspace.from_config(auth=cli_auth)
ws = Workspace.get(
        name=workspace_name,
        subscription_id=subscription_id,
        resource_group=resource_group,
    )


In [4]:
env = Environment.get(workspace=ws, name="AzureML-Minimal")
env

{
    "name": "AzureML-Minimal",
    "version": "20",
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "python": {
        "userManagedDependencies": false,
        "interpreterPath": "python",
        "condaDependenciesFile": null,
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "conda-forge"
            ],
            "dependencies": [
                "python=3.6.2",
                {
                    "pip": [
                        "azureml-core==1.3.0",
                        "azureml-defaults==1.3.0"
                    ]
                }
            ],
            "name": "azureml_f851a99ad48fd8bf6717f441e3edf254"
        }
    },
    "docker": {
        "enabled": false,
        "baseImage": "mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04",
        "baseDockerfile": null,
        "sharedVolumes": true,
        "shmSize": null,
        "arguments": [],
        "baseI

In [5]:
envs = Environment.list(workspace=ws)

for env in envs:
    if env.startswith("AzureML"):
        print("Name",env)
        print("packages", envs[env].python.conda_dependencies.serialize_to_string())

Name AzureML-Chainer-5.1.0-GPU
packages channels:
- conda-forge
dependencies:
- python=3.6.2
- pip:
  - azureml-core==1.3.0
  - azureml-defaults==1.3.0
  - azureml-telemetry==1.3.0
  - azureml-train-restclients-hyperdrive==1.3.0
  - azureml-train-core==1.3.0
  - chainer==5.1.0
  - cupy-cuda90==5.1.0
  - mpi4py==3.0.0
name: azureml_4fbbe2f12ddfc544edcea4b265a579f8

Name AzureML-Designer-R
packages channels:
- conda-forge
dependencies:
- python=3.6.8
- r-caret=6.0
- r-catools=1.17.1
- r-cluster=2.1.0
- r-dplyr=0.8.5
- r-e1071=1.7
- r-forcats=0.5.0
- r-glmnet=2.0
- r-matrix=1.2
- r-mgcv=1.8
- r-nlme=3.1
- r-nnet=7.3
- r-plyr=1.8.6
- r-randomforest=4.6
- r-reticulate=1.12
- r-rocr=1.0
- r-rpart=4.1
- r-stringr=1.4.0
- r-tidyverse=1.2.1
- r-timedate=3043.102
- r-tseries=0.10
- r=3.5.1
- pip:
  - azureml-designer-classic-modules==0.0.119
name: azureml_06b1828acac23daf680a3b3d3c84be0e

Name AzureML-Scikit-learn-0.20.3
packages channels:
- conda-forge
dependencies:
- python=3.6.2
- pip:
  - az

In [6]:
from azureml.core.environment import Environment
Environment(name="arimaenv")

{
    "name": "arimaenv",
    "version": null,
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "python": {
        "userManagedDependencies": false,
        "interpreterPath": "python",
        "condaDependenciesFile": null,
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "name": "project_environment",
            "dependencies": [
                "python=3.6.2",
                {
                    "pip": [
                        "azureml-defaults"
                    ]
                }
            ],
            "channels": [
                "conda-forge"
            ]
        }
    },
    "docker": {
        "enabled": false,
        "baseImage": "mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04",
        "baseDockerfile": null,
        "sharedVolumes": true,
        "shmSize": "2g",
        "arguments": [],
        "baseImageRegistry": {
            "address": null,
            "username": null,
      

In [17]:
# From a Conda specification file
arimaenv = Environment.from_conda_specification(name = "arimaenv", file_path = "./scripts/scoring/conda_dependencies.yml")

# From a pip requirements file
#myenv = Environment.from_pip_requirements(name = "myenv", file_path = "path-to-pip-requirements-file")

In [18]:
arimaenv

{
    "name": "arimaenv",
    "version": null,
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "python": {
        "userManagedDependencies": false,
        "interpreterPath": "python",
        "condaDependenciesFile": null,
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "name": "project_environment",
            "dependencies": [
                "python=3.6.2",
                "psutil=5.3",
                {
                    "pip": [
                        "azureml-sdk[notebooks]",
                        "azureml-defaults",
                        "scipy==1.0.0",
                        "scikit-learn==0.19.1",
                        "pandas==0.23.1",
                        "numpy==1.14.5",
                        "statsmodels",
                        "matplotlib",
                        "seaborn"
                    ]
                }
            ]
        }
    },
    "docker": {
        "enabled": false,

In [19]:
arimaenv.register(workspace=ws)

{
    "name": "arimaenv",
    "version": "2",
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "python": {
        "userManagedDependencies": false,
        "interpreterPath": "python",
        "condaDependenciesFile": null,
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "dependencies": [
                "python=3.6.2",
                "psutil=5.3",
                {
                    "pip": [
                        "azureml-sdk[notebooks]",
                        "azureml-defaults",
                        "scipy==1.0.0",
                        "scikit-learn==0.19.1",
                        "pandas==0.23.1",
                        "numpy==1.14.5",
                        "statsmodels",
                        "matplotlib",
                        "seaborn"
                    ]
                }
            ],
            "name": "azureml_8cca4fc2bfc01907bd93cb07800ada55"
        }
    },
    "docker": {
     

In [20]:
restored_environment = Environment.get(workspace=ws,name="arimaenv",version="1")
restored_environment

{
    "name": "arimaenv",
    "version": "1",
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "python": {
        "userManagedDependencies": false,
        "interpreterPath": "python",
        "condaDependenciesFile": null,
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "dependencies": [
                "python=3.6.2",
                "psutil=5.3",
                {
                    "pip": [
                        "azureml-sdk[notebooks]",
                        "scipy==1.0.0",
                        "scikit-learn==0.19.1",
                        "pandas==0.23.1",
                        "numpy==1.14.5",
                        "statsmodels",
                        "matplotlib",
                        "seaborn"
                    ]
                }
            ],
            "name": "azureml_aa61f2081e16c2169fd3245a49c4f755"
        }
    },
    "docker": {
        "enabled": false,
        "baseImage": "m

In [21]:
# Creates the environment inside a Docker container.
arimaenv.docker.enabled = True

In [22]:
try:
    with open("./configuration/model.json") as f:
        config = json.load(f)
except:
    print("No new model to register thus no need to create new scoring image")
    # raise Exception('No new model to register as production model perform better')
    sys.exit(0)

model_name = config["model_name"]
model_version = config["model_version"]

model_list = Model.list(workspace=ws)
model, = (m for m in model_list if m.version == model_version and m.name == model_name)
print(
    "Model picked: {} \nModel Description: {} \nModel Version: {}".format(
        model.name, model.description, model.version
    )
)

Model picked: arima_notebook_model.pkl 
Model Description: Time series forecasting model for Boston robberies dataset 
Model Version: 1


In [23]:
model

Model(workspace=Workspace.create(name='ShivaMLservice', subscription_id='46668180-b0ad-4a49-bed9-88f16f315dce', resource_group='MLGroup'), name=arima_notebook_model.pkl, id=arima_notebook_model.pkl:1, version=1, tags={'area': 'robberies', 'type': 'forecasting', 'run_id': 'arima-localrun_1587671856_2c6a35ac'}, properties={})

In [24]:
# Combine scoring script & environment in Inference configuration
inference_config = InferenceConfig(entry_script="./scripts/scoring/score.py", environment=arimaenv)

In [25]:
deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1,
    tags={"area": "robberies", "type": "forecasting"},
    description="forecasting of Boston robberies using ARIMA",
)

In [28]:
service = Model.deploy(
    workspace = ws,
    name = "arima-ws-env",
    models = [model],
    inference_config = inference_config,
    deployment_config = deployment_config)

In [29]:
service.wait_for_deployment()

ACI service creation operation finished, operation "Succeeded"
